# 1 Load Data

In [ ]:
import torch
from rich.jupyter import print
X = torch.load('data/X.tensor')
X.shape

# 2 Load Model

In [41]:
n_layer = 12
n_head = 12
n_embd = 768
dropout = 0.1
bias = False

model_args =  dict(n_layer=n_layer, n_head=n_head, n_embd=n_embd, block_size=1024,
                  bias=bias, vocab_size=64, dropout=dropout)

from model import GPTConfig, GPT
gptconf = GPTConfig(**model_args)
model = GPT(gptconf).to('cpu')
print("预测词表大小", model.config.vocab_size) # BPE词表

number of parameters: 85.00M


预测词表大小 64

# 3 Greedy Generation

In [36]:
# 推2个token
from torch .nn import functional as F
idx =  X[1,:10].reshape(1,10)
print("预测词表大小", model.config.vocab_size) # BPE词表
print("输入X:", idx)
print("输入X长度:", idx.shape)
for _ in range(2):
    print(f'----------{_+1} token----------')
    logits, _ = model(idx)
    print('输出Logits:', logits.shape)
    probs = F.softmax(logits, dim=-1)
    print('输出Probs:', probs.shape)
    idx_next = torch.argmax(probs, dim=-1)
    print('输出下一个Token:', idx_next.shape)
    print('token:', idx_next)
    
    idx = torch.cat((idx, idx_next), dim=-1)
    print("当前的token长度:", len(idx[0]))
    print("当前的token序列:", idx)
    
    

预测词表大小 64

输入X: tensor([[ 0, 13, 52, 42,  1, 26, 43, 56, 53,  1]])

输入X长度:
torch.Size([1, 10])

----------1 token----------

输出Logits:
torch.Size([1, 1, 64])

...

输出Probs:
torch.Size([1, 1, 64])

输出下一个Token:
torch.Size([1, 1])

token: tensor([[38]])

当前的token长度: 12

当前的token序列: tensor([[ 0, 13, 52, 42,  1, 26, 43, 56, 53,  1,  8, 38]])

In [38]:
torch.set_printoptions(precision=4)
torch.set_printoptions(sci_mode=False)
torch.set_printoptions(linewidth=100)
'''
torch.set_printoptions 是 PyTorch 中用来设置张量打印选项的函数。每个参数的作用如下：

precision: 设置浮点数的打印精度。例如，precision=4 表示打印浮点数时保留四位小数。

sci_mode: 控制科学计数法的使用。当设置为 False 时，张量将以普通数字格式打印，而不是科学计数法。默认情况下，如果数字太大或太小，PyTorch 会自动使用科学计数法。

linewidth: 设置每行的最大字符数。超过这个长度的行将会被换行。设置为 linewidth=100 表示每行最多打印 100 个字符。

这些选项有助于提高打印输出的可读性，尤其在处理较大的张量时。
'''

'\ntorch.set_printoptions 是 PyTorch 中用来设置张量打印选项的函数。每个参数的作用如下：\n\nprecision: 设置浮点数的打印精度。例如，precision=4 表示打印浮点数时保留四位小数。\n\nsci_mode: 控制科学计数法的使用。当设置为 False 时，张量将以普通数字格式打印，而不是科学计数法。默认情况下，如果数字太大或太小，PyTorch 会自动使用科学计数法。\n\nlinewidth: 设置每行的最大字符数。超过这个长度的行将会被换行。设置为 linewidth=100 表示每行最多打印 100 个字符。\n\n这些选项有助于提高打印输出的可读性，尤其在处理较大的张量时。\n'

## 3.1 Top-K GPT-2

In [81]:
temperature = 1.0
top_k = 5
print(model.config.vocab_size) # BPE词表
print("input_ids", idx)

64

input_ids tensor([[ 0, 13, 52, 42,  1, 26, 43, 56, 53,  1, 42, 36, 36, 36, 36,  7, 25, 46, 53, 53, 53, 59]])

In [85]:
from torch.nn import functional as F

idx = X[1,:10].reshape(1, 10)
print(idx.shape)
for _ in range(10):
    idx_cond = idx if idx.size(1) <= model.config.block_size else idx[:,-model.config.block_size]
    
    logits, _ = model(idx_cond)
    print('logits:',logits.shape)
    
    print("no - 1", logits[:, -1, :].shape)  
    logits = logits[:, -1, :] / temperature
    
    if top_k:
        # 找到top k更高的概率
        v, i = torch.topk(logits, min(top_k, logits.size(1)))
        print('top_k v:', v, v[:, -1])
        print('i', i)
        
        logits[logits < v[:, -1]] = -float('Inf')
        print('top_k logits', logits.shape)
        print('logits', logits)
    
    # softmax
    probs = F.softmax(logits, dim=-1)
    print('probs', probs.shape)
    print(probs)
    
    #sample
    idx_next = torch.multinomial(probs, num_samples=1)
    print('idx_next', idx_next)
    
    idx = torch.cat((idx, idx_next), dim=-1)
    print('new_idx', idx.shape)
    
    print("generate: length :", len(idx[0]))
    print("generate:", idx)

torch.Size([1, 10])

logits:
torch.Size([1, 1, 64])

no - 1
torch.Size([1, 64])

top_k v: tensor([[1.9979, 1.2121, 1.1881, 1.1132, 1.0095]], grad_fn=<TopkBackward0>) tensor([1.0095], 
grad_fn=<SelectBackward0>)

i tensor([[ 1, 42, 28, 36, 37]])

...

tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.1586, 0.2074, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.2261, 0.0000, 0.1868, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.2212, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000]],
       grad_fn=<SoftmaxBackward0>)

idx_next tensor([[8]])

new_idx
torch.Size([1, 20])

generate: length : 20

generate: tensor([[ 0, 13, 52, 42,  1, 26, 43, 56, 53,  1,  1, 23, 28,  0,  0, 27, 20, 45, 25,  8]])

In [90]:
idx = X[1,:10].reshape(1, 10)
model.generate(idx, 10, 1.0, 5) # temparature 1.0 ,  Top_K None 

tensor([[ 0, 13, 52, 42,  1, 26, 43, 56, 53,  1, 59, 36,  8,  0,  0,  7,  7, 25,  1, 43]])

## 3.2 Sample

In [116]:
dict_map = {0:'🐯', 1:'🐷', 2:'🐱'}
probs = torch.tensor([0.7,0.2,0.1])
print('probs', probs)

print('根据概率选next token')
for _ in range(10):
    sample = torch.multinomial(probs, num_samples=1)[0]
    print(f'第{_+1}次抽样结果为{dict_map[int(sample)]}')

probs tensor([0.7000, 0.2000, 0.1000])

根据概率选next token

第1次抽样结果为🐯

第2次抽样结果为🐯

第3次抽样结果为🐯

...

第6次抽样结果为🐯

第7次抽样结果为🐱

第8次抽样结果为🐷

第9次抽样结果为🐷

第10次抽样结果为🐯

## 3.3 Sample with temperature

In [123]:
# temp > 1，概率分布更平滑，temp < 1，概率分布更陡峭
temperature = 0.1
dict_map = {0:'🐯', 1:'🐷', 2:'🐱'}
probs = torch.tensor([0.7,0.2,0.1])
print('probs', probs)
probs = probs / temperature
print('probs with temperature', probs)

print('根据概率选next token')
for _ in range(10):
    sample = torch.multinomial(probs, num_samples=1)[0]
    print(f'第{_+1}次抽样结果为{dict_map[int(sample)]}')

probs tensor([0.7000, 0.2000, 0.1000])

probs with temperature tensor([7., 2., 1.])

根据概率选next token

第1次抽样结果为🐯

第2次抽样结果为🐯

...

第6次抽样结果为🐯

第7次抽样结果为🐷

第8次抽样结果为🐯

第9次抽样结果为🐷

第10次抽样结果为🐯

## 3.4  Top K

In [133]:
T = 0.1
top_k = 2
dict_map = {0:'🐯', 1:'🐷', 2:'🐱'}
probs = torch.tensor([0.7,0.2,0.1])
probs /= T
print('probs/T', probs)
probs = F.softmax(probs, dim=-1)
print('softmax_probs', probs)
probs, _ = torch.topk(probs, top_k)
print('topk_probs', probs)
probs = F.softmax(probs, dim=-1)
print('topk_softmax_probs', probs)

print('根据概率选next token')
for _ in range(10):
    sample = torch.multinomial(probs, num_samples=1)
    print(f'第{_+1}次抽样结果为{dict_map[int(sample)]}')

probs/T tensor([7., 2., 1.])

softmax_probs tensor([0.9909, 0.0067, 0.0025])

topk_probs tensor([0.9909, 0.0067])

topk_softmax_probs tensor([0.7279, 0.2721])

根据概率选next token

...

第6次抽样结果为🐯

第7次抽样结果为🐯

第8次抽样结果为🐷

第9次抽样结果为🐯

第10次抽样结果为🐯

In [145]:
idx = X[1,:10].reshape(1, 10) 
print(idx)
print(idx.shape)

tensor([[ 0, 13, 52, 42,  1, 26, 43, 56, 53,  1]])

torch.Size([1, 10])

## 3.5 repetition penalty

In [158]:
penalty = 2.0
for _ in range(256):
    logits, _ = model(idx)
    print(logits.shape)
    print(logits)
    logits = logits[:, -1, :]
    original_logits = logits.clone()
    
    print(logits.shape)
    logits_idx = torch.gather(logits, 1, idx)
    print(logits_idx)
    logits_idx = torch.where(logits_idx < 0, logits_idx * penalty, logits_idx / penalty).clone()
    print(logits_idx)
    logits = logits.scatter_(1, idx, logits_idx)
    print(logits)

    probs = F.softmax(logits, dim=-1)
    print(probs)
    idx_next = torch.multinomial(probs, num_samples=1)
    idx = torch.cat((idx, idx_next), dim=-1)

    break

original_logits - logits

torch.Size([1, 1, 64])

tensor([[[ 0.2973,  0.9324, -0.1599, -0.3171,  0.0441, -0.4419,  0.6607,  0.1825,  0.4705, -0.2182,
           0.6306,  0.1088, -0.2865,  0.0636, -0.3344,  0.4229,  1.1104,  0.3696,  0.0176, -0.8602,
           0.1445, -0.1206,  0.1589,  0.3093,  0.0166,  0.6258, -0.1401,  0.3005,  0.1242, -0.7279,
           0.1594,  0.1922, -0.1273,  0.4480, -1.0523, -0.2502,  1.2481,  1.3518,  0.3286, -0.8358,
          -0.8232, -0.4944, -0.0551,  0.4902,  0.0892,  1.1445,  0.8095, -1.1612, -0.0973,  0.4467,
           0.2858,  0.1293, -0.3478,  0.1086, -1.2691, -0.8461, -0.5049,  0.1477,  0.3072,  0.1215,
          -0.8545,  0.1165, -0.8258,  0.2118]]], grad_fn=<UnsafeViewBackward0>)

torch.Size([1, 64])

tensor([[ 0.2973,  0.0636, -0.3478, -0.0551,  0.9324, -0.1401,  0.4902, -0.5049,  0.1086,  0.9324,
          0.8095]], grad_fn=<GatherBackward0>)

tensor([[ 0.1487,  0.0318, -0.6955, -0.1103,  0.4662, -0.2801,  0.2451, -1.0097,  0.0543,  0.4662,
          0.4048]], grad_fn=<CloneBackward0>)

tensor([[ 0.1487,  0.4662, -0.1599, -0.3171,  0.0441, -0.4419,  0.6607,  0.1825,  0.4705, -0.2182,
          0.6306,  0.1088, -0.2865,  0.0318, -0.3344,  0.4229,  1.1104,  0.3696,  0.0176, -0.8602,
          0.1445, -0.1206,  0.1589,  0.3093,  0.0166,  0.6258, -0.2801,  0.3005,  0.1242, -0.7279,
          0.1594,  0.1922, -0.1273,  0.4480, -1.0523, -0.2502,  1.2481,  1.3518,  0.3286, -0.8358,
         -0.8232, -0.4944, -0.1103,  0.2451,  0.0892,  1.1445,  0.4048, -1.1612, -0.0973,  0.4467,
          0.2858,  0.1293, -0.6955,  0.0543, -1.2691, -0.8461, -1.0097,  0.1477,  0.3072,  0.1215,
         -0.8545,  0.1165, -0.8258,  0.2118]], grad_fn=<AsStridedBackward0>)

tensor([[0.0156, 0.0214, 0.0114, 0.0098, 0.0140, 0.0086, 0.0260, 0.0161, 0.0215, 0.0108, 0.0252,
         0.0150, 0.0101, 0.0139, 0.0096, 0.0205, 0.0407, 0.0194, 0.0137, 0.0057, 0.0155, 0.0119,
         0.0157, 0.0183, 0.0136, 0.0251, 0.0101, 0.0181, 0.0152, 0.0065, 0.0157, 0.0163, 0.0118,
         0.0210, 0.0047, 0.0104, 0.0467, 0.0519, 0.0186, 0.0058, 0.0059, 0.0082, 0.0120, 0.0171,
         0.0147, 0.0421, 0.0201, 0.0042, 0.0122, 0.0210, 0.0179, 0.0153, 0.0067, 0.0142, 0.0038,
         0.0058, 0.0049, 0.0156, 0.0182, 0.0152, 0.0057, 0.0151, 0.0059, 0.0166]],
       grad_fn=<SoftmaxBackward0>)

tensor([[0.1487, 0.4662, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0318, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.1401, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0551, 0.2451,
         0.0000, 0.0000, 0.4048, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.3478, 0.0543, 0.0000,
         0.0000, 0.5049, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000]],
       grad_fn=<SubBackward0>)

# 4. Huggingface Transformer Generate

In [159]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

torch_device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained("gpt2")

model_GPT = AutoModelForCausalLM.from_pretrained("gpt2", pad_token_id=tokenizer.eos_token_id).to(torch_device)

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [160]:
print(tokenizer)

GPT2TokenizerFast(name_or_path='gpt2', vocab_size=50257, model_max_length=1024, is_fast=True, padding_side='right',
truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': 
'<|endoftext|>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
        50256: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, 
special=True),
}

In [173]:
text = "pneumonoultramicroscopicsilicovolcanoconiosis "

inputs = tokenizer(text, return_tensors='pt').get('input_ids')
print(inputs)
print(inputs[0,0])
print(tokenizer.decode(inputs[0,0]))
print(tokenizer.decode(inputs[0]))

tensor([[   79, 25668,   261, 25955,   859,  2500,  1416,   404,   873, 41896,   709,   349,  5171,
         36221, 42960,   220]])

tensor(79)

p

pneumonoultramicroscopicsilicovolcanoconiosis

In [175]:
inputs = tokenizer("hello world", return_tensors='pt')
inputs1 = tokenizer(" hello world", return_tensors='pt')
print(inputs)
print(inputs1)

{'input_ids': tensor([[31373,   995]]), 'attention_mask': tensor([[1, 1]])}

{'input_ids': tensor([[23748,   995]]), 'attention_mask': tensor([[1, 1]])}

## 4.1 Generate Beam search (num_beams, top_k, top_p, temperature, repetition_penalty)
### 4.1.1 activate beam search and early_stopping

In [199]:
model_inputs = tokenizer('I enjoy walking with my cute dog', return_tensors='pt').to(torch_device)

beam_output = model_GPT.generate(
    **model_inputs,
    max_new_tokens=40,
    num_beams=5,
    early_stopping=True
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(beam_output[0]))
help(model.generate)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Output:
----------------------------------------------------------------------------------------------------

I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with him again.

I'm not sure if I'll ever be able to walk with him again. I'm not sure

Help on method generate in module model:

generate(idx, max_new_tokens, temperature=1.0, top_k=None) method of model.GPT instance
    Take a conditioning sequence of indices idx (LongTensor of shape (b,t)) and complete
    the sequence max_new_tokens times, feeding the predictions back into the model each time.
    Most likely you'll want to make sure to be in model.eval() mode of operation for this.



### 4.1.2 set no_repeat_ngram_size to 2

In [198]:
## Generate Beam Searching
# set no_repeat_ngram_size to 2
beam_output = model_GPT.generate(
    **model_inputs,
    max_new_tokens=40,
    num_beams=5,
    no_repeat_ngram_size=2,
    early_stopping=True
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(beam_output[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Output:
----------------------------------------------------------------------------------------------------

I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with him again.

I've been thinking about this for a while now, and I think it's time for me to

### 4.1.3 set return_num_sequences > 1

In [206]:

beam_outputs = model_GPT.generate(
    **model_inputs,
    max_new_tokens=40,
    num_beams=5,
    no_repeat_ngram_size=2,
    num_return_sequences=5,
    early_stopping=True
)

# now we have 3 output sequences
print("Output:\n" + 100 * '-')
for i, beam_output in enumerate(beam_outputs):
    print(f'{i}:{tokenizer.decode(beam_output, skip_special_tokens=True)}')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Output:
----------------------------------------------------------------------------------------------------

0:I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with him again.

I've been thinking about this for a while now, and I think it's time for me to

1:I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with her again.

I've been thinking about this for a while now, and I think it's time for me to

2:I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with him again.

I've been thinking about this for a while now, and I think it's a good idea to

3:I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with him again.

I've been thinking about this for a while now, and I think it's time to take a

4:I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with him again.

I've been thinking about this for a while now, and I think it's a good idea.

### 4.1.4 sampling

In [215]:
from transformers import set_seed

set_seed(13)
sample_output = model_GPT.generate(
    **model_inputs,
    max_new_tokens=40,
    do_sample=True,
    top_k=0#
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Output:
----------------------------------------------------------------------------------------------------

I enjoy walking with my cute dog, but he has neglected me after about 5 weeks because 'hee I miss him.' Poor man!" 
Carol Leslie)

Thought it's funny, but he looks unchanged! Coulter said, "

### 4.1.5 sampling temparature

In [216]:
sample_output = model_GPT.generate(
    **model_inputs,
    max_new_tokens=40,
    do_sample=True,
    top_k=0,
    temperature=0.6,
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Output:
----------------------------------------------------------------------------------------------------

I enjoy walking with my cute dog and I have a lot of fun doing it. It's fun to be around strangers and it's fun to 
be around people. I really like to be around people.

What is your favorite

### 4.1.6 top-k sampling

In [217]:
set_seed(42)

# set top_k to 50
sample_output = model_GPT.generate(
    **model_inputs,
    max_new_tokens=40,
    do_sample=True,
    top_k=50
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Output:
----------------------------------------------------------------------------------------------------

I enjoy walking with my cute dog but what I love about being a dog is I see a beautiful pet being cared for – I 
love having the opportunity to see her every day so I feel very privileged to have been able to help this

### 4.1.7 Top-p (nucleus) sampling

In [218]:
set_seed(42)

# set top_k to 50
sample_output = model_GPT.generate(
    **model_inputs,
    max_new_tokens=40,
    do_sample=True,
    top_p=0.92,
    top_k=50
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Output:
----------------------------------------------------------------------------------------------------

I enjoy walking with my cute dog but sometimes I get nervous when she is around. I've been told that this is 
because I am an overweight man and I like to be around her. It's nice to know that I have this

In [221]:
sample_outputs = model_GPT.generate(
    **model_inputs,
    max_new_tokens=40,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    num_return_sequences=3,
    repetition_penalty=1.2
)

print("Output:\n" + 100 * '-')
for i, sample_output in enumerate(sample_outputs):
  print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Output:
----------------------------------------------------------------------------------------------------

0: I enjoy walking with my cute dog after school! My mom would love to play and see how I did at home. We went on a
hike in Maine together last May while they were both out playing Pokémon Go, but instead we stayed

1: I enjoy walking with my cute dog when we are out," said Marylou.
"She's got a great attitude, she listens and loves taking care of herself as well."

2: I enjoy walking with my cute dog and taking time to play Pokémon. It seems like a good thing because the only 
one who can go back for an hour is him right?"
"But no," she said as though it were serious

# 5 Decoder loss

In [222]:
import torch
import math

In [223]:
batch_size = 1  # 一句话

x = torch.randn(batch_size, 4, 512) #input :  batch_size, length, embd_dim
y = torch.randint(low=0, high=32000, size=(batch_size, 4), dtype=torch.long)

print(x.shape)
print(y.shape)
print(y)

torch.Size([1, 4, 512])

torch.Size([1, 4])

tensor([[25133, 25687, 18186, 24893]])

In [224]:
q = torch.randn(512, 512)  
k = torch.randn(512, 512)
v = torch.randn(512, 512)
o = torch.randn(512, 512)

mask=torch.tril(torch.ones(1, 4, 4))
print(mask)

tensor([[[1., 0., 0., 0.],
         [1., 1., 0., 0.],
         [1., 1., 1., 0.],
         [1., 1., 1., 1.]]])

In [225]:
# scaled dot produc attention 
Q, K, V = x@q, x@k, x@v
scores = Q@K.transpose(1,2) / math.sqrt(512)
scores = scores.masked_fill(mask==0, float(-10000.0))
weights = F.softmax(scores, dim=-1)
attn = weights @ V
attn = attn @ o
attn.shape

torch.Size([1, 4, 512])

In [230]:
# mlp
mlp_up = torch.randn(512, 1024)
mlp_down = torch.randn(1024, 512)

mlp = attn @ mlp_up @ mlp_down
mlp.shape

torch.Size([1, 4, 512])

In [233]:
lm_head = torch.randn(512, 32000)
logits = mlp @ lm_head
logits.shape

torch.Size([1, 4, 32000])

In [235]:
probs = torch.softmax(logits, dim=2)

print(probs.shape)

torch.Size([1, 4, 32000])

In [256]:
# 计算交叉熵损失
print(probs.shape)
print(y.shape)
print('prob:',probs.view(-1, probs.size(-1)).shape)
print('y:',y.view(-1).shape)
loss = F.cross_entropy(probs.view(-1, probs.size(-1)), y.view(-1))
print('loss:', loss)

torch.Size([1, 4, 32000])

torch.Size([1, 4])

prob:
torch.Size([4, 32000])

y:
torch.Size([4])

loss: tensor(10.3736)